In [1]:
import torch_geometric as tg

In [67]:
tg_dataset_path = "/mnt/sd1/torch_geometric/indoor3d_sem_seg_hdf5_data"

In [33]:
dataset = tg.datasets.S3DIS(tg_dataset_path, test_area=5)

In [1]:
import omegaconf

In [2]:
tp3d_dataset_path = "/home/coder/workspace/data2/torch_point3d/"

In [3]:
cfg = omegaconf.OmegaConf.load("s3disfused.yaml")
dataset_cfg = cfg.data
dataset_cfg.dataroot = tp3d_dataset_path

In [4]:
import os, sys
sys.path.append("../")

In [5]:
from torch_points3d.datasets.segmentation.s3dis import S3DISFusedDataset

In [6]:
dataset = S3DISFusedDataset(cfg.data)

Processing...
Done!


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
data = dataset.train_dataset[4]

AttributeError: 'NoneType' object has no attribute 'values'

In [59]:
import torchpcp
from torchpcp.utils.pytorch_tools import t2n
from torchpcp.utils.io.ply import write_pc

In [60]:
path2saveply = "s3disfuseddataset_4.ply"
# dataset.to_ply(data.pos, data.y, path2saveply)
pos = t2n(data.pos)
write_pc(path2saveply, pos)


In [42]:
from torch_points3d.datasets.segmentation.s3dis import S3DIS1x1Dataset

In [51]:
cfg2 = omegaconf.OmegaConf.load("s3dis1x1.yaml")
dataset_cfg2 = cfg2.data
dataset_cfg2.dataroot = tp3d_dataset_path

In [52]:
dataset2 = S3DIS1x1Dataset(dataset_cfg2)

In [55]:
data2 = dataset2.train_dataset[4]

In [61]:
path2saveply2 = "s3disfuseddataset_4_2.ply"
# dataset.to_ply(data.pos, data.y, path2saveply)
pos2 = t2n(data2.pos)
write_pc(path2saveply2, pos2)
data2

Data(pos=[4096, 3], x=[4096, 6], y=[4096])

In [91]:
import os
import os.path as osp
import shutil

import h5py
import torch
from torch_geometric.data import (InMemoryDataset, Data, download_url,
                                  extract_zip)

class S3DIS(InMemoryDataset):
    r"""The (pre-processed) Stanford Large-Scale 3D Indoor Spaces dataset from
    the `"3D Semantic Parsing of Large-Scale Indoor Spaces"
    <http://buildingparser.stanford.edu/images/3D_Semantic_Parsing.pdf>`_
    paper, containing point clouds of six large-scale indoor parts in three
    buildings with 12 semantic elements (and one clutter class).

    Args:
        root (string): Root directory where the dataset should be saved.
        test_area (int, optional): Which area to use for testing (1-6).
            (default: :obj:`6`)
        train (bool, optional): If :obj:`True`, loads the training dataset,
            otherwise the test dataset. (default: :obj:`True`)
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
        pre_filter (callable, optional): A function that takes in an
            :obj:`torch_geometric.data.Data` object and returns a boolean
            value, indicating whether the data object should be included in the
            final dataset. (default: :obj:`None`)
    """

    url = ('https://shapenet.cs.stanford.edu/media/'
           'indoor3d_sem_seg_hdf5_data.zip')

    def __init__(self,
                 root,
                 test_area=6,
                 train=True,
                 transform=None,
                 pre_transform=None,
                 pre_filter=None):
        assert test_area >= 1 and test_area <= 6
        self.test_area = test_area
        super(S3DIS, self).__init__(root, transform, pre_transform, pre_filter)
        path = self.processed_paths[0] if train else self.processed_paths[1]
        self.data, self.slices = torch.load(path)

    @property
    def raw_file_names(self):
        return ['all_files.txt', 'room_filelist.txt']

    @property
    def processed_file_names(self):
        test_area = self.test_area
        return ['{}_{}.pt'.format(s, test_area) for s in ['train', 'test']]

    def download(self):
        path = download_url(self.url, self.root)
        extract_zip(path, self.root)
        os.unlink(path)
        shutil.rmtree(self.raw_dir)
        name = self.url.split(os.sep)[-1].split('.')[0]
        os.rename(osp.join(self.root, name), self.raw_dir)

    def process(self):
        with open(self.raw_paths[0], 'r') as f:
            filenames = [x.split('/')[-1] for x in f.read().split('\n')[:-1]]

        with open(self.raw_paths[1], 'r') as f:
            rooms = f.read().split('\n')[:-1]

        xs, ys = [], []
        gs = []
        for filename in filenames:
            f = h5py.File(osp.join(self.raw_dir, filename))
            print(f.keys())
            xs += torch.from_numpy(f['data'][:]).unbind(0)
            ys += torch.from_numpy(f['label'][:]).to(torch.long).unbind(0)
            print(torch.max(ys[-1]))

        test_area = 'Area_{}'.format(self.test_area)
        train_data_list, test_data_list = [], []
        for i, (x, y) in enumerate(zip(xs, ys)):
            data = Data(pos=x[:, :3], x=x[:, 3:], y=y)

            if self.pre_filter is not None and not self.pre_filter(data):
                continue
            if self.pre_transform is not None:
                data = self.pre_transform(data)

            if test_area not in rooms[i]:
                train_data_list.append(data)
            else:
                test_data_list.append(data)

        torch.save(self.collate(train_data_list), self.processed_paths[0])
        torch.save(self.collate(test_data_list), self.processed_paths[1])



In [92]:
dataset3 = S3DIS(tg_dataset_path)

Processing...
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(8)
<KeysViewHDF5 ['data', 'label']>
tensor(8)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(1)
<KeysViewHDF5 ['data', 'label']>
tensor(6)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(7)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(2)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(6)
<KeysViewHDF5 ['data', 'label']>
tensor(7)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'label']>
tensor(10)
<KeysViewHDF5 ['data', 'label']>
tensor(1)
<KeysViewHDF5 ['data', 'label']>
tensor(12)
<KeysViewHDF5 ['data', 'lab

KeyboardInterrupt: 